In [0]:
invoices = sqlContext.read \
    .csv("/databricks-datasets/online_retail/data-001/data.csv", header=True).rdd

print(invoices.take(5))

[Row(InvoiceNo='536365', StockCode='85123A', Description='WHITE HANGING HEART T-LIGHT HOLDER', Quantity='6', InvoiceDate='12/1/10 8:26', UnitPrice='2.55', CustomerID='17850', Country='United Kingdom'), Row(InvoiceNo='536365', StockCode='71053', Description='WHITE METAL LANTERN', Quantity='6', InvoiceDate='12/1/10 8:26', UnitPrice='3.39', CustomerID='17850', Country='United Kingdom'), Row(InvoiceNo='536365', StockCode='84406B', Description='CREAM CUPID HEARTS COAT HANGER', Quantity='8', InvoiceDate='12/1/10 8:26', UnitPrice='2.75', CustomerID='17850', Country='United Kingdom'), Row(InvoiceNo='536365', StockCode='84029G', Description='KNITTED UNION FLAG HOT WATER BOTTLE', Quantity='6', InvoiceDate='12/1/10 8:26', UnitPrice='3.39', CustomerID='17850', Country='United Kingdom'), Row(InvoiceNo='536365', StockCode='84029E', Description='RED WOOLLY HOTTIE WHITE HEART.', Quantity='6', InvoiceDate='12/1/10 8:26', UnitPrice='3.39', CustomerID='17850', Country='United Kingdom')]


##Q1: Which customer in the dataset has spent the most on products?

In [0]:
mappedJob = invoices.map(lambda invoice: (invoice[6], (float(invoice[3]) * float(invoice[5])))) # Output pair RDD is (customer_number, total_revenue)
print(mappedJob.take(5))
reducedJob = mappedJob.reduceByKey(lambda x,y:x+y)
print(reducedJob.take(5))

[('17850', 15.299999999999999), ('17850', 20.34), ('17850', 22.0), ('17850', 20.34), ('17850', 20.34)]
[('13047', 363.6800000000001), ('12583', 1586.0199999999998), ('13748', 204.0), ('14688', 747.2), ('15311', 6047.090000000002)]


In [0]:
Sort = reducedJob.filter(lambda invoice: invoice[0] is not None).sortBy(lambda sort: -sort[1])
print(Sort.take(5))
MostSpend = Sort.toDF(['Customer','Spend']) 
display(MostSpend.take(10))

[('18102', 27834.61), ('14646', 27008.25999999999), ('15749', 22998.4), ('15061', 22045.21000000001), ('17450', 20649.039999999994)]


Customer,Spend
18102,27834.61
14646,27008.25999999999
15749,22998.4
15061,22045.21000000001
17450,20649.039999999994
14156,17089.42
16029,14730.150000000001
17511,10459.3
14911,10004.880000000001
13089,9530.390000000001


##Q2:	What is the product description for the best selling product in the dataset?  will define "Best Selling" as the product with the highest quantity sold.

In [0]:
mappedJob1 = invoices.map(lambda BestS: (BestS[2], (float(BestS[3] ))))
reducedJob1 = mappedJob1.reduceByKey(lambda x,y:x+y).sortBy(lambda sort: -sort[1])

print(mappedJob1.take(5))
print(reducedJob1.take(5))
BestSell = reducedJob1.toDF(['Product Description','Quantity '])
display(BestSell.take(5))

[('WHITE HANGING HEART T-LIGHT HOLDER', 6.0), ('WHITE METAL LANTERN', 6.0), ('CREAM CUPID HEARTS COAT HANGER', 8.0), ('KNITTED UNION FLAG HOT WATER BOTTLE', 6.0), ('RED WOOLLY HOTTIE WHITE HEART.', 6.0)]
[('WHITE HANGING HEART T-LIGHT HOLDER', 8189.0), ('PACK OF 72 RETROSPOT CAKE CASES', 6634.0), ('WORLD WAR 2 GLIDERS ASSTD DESIGNS', 6109.0), ('HAND WARMER BABUSHKA DESIGN', 4060.0), ('HEART OF WICKER SMALL', 4047.0)]


Product Description,Quantity
WHITE HANGING HEART T-LIGHT HOLDER,8189.0
PACK OF 72 RETROSPOT CAKE CASES,6634.0
WORLD WAR 2 GLIDERS ASSTD DESIGNS,6109.0
HAND WARMER BABUSHKA DESIGN,4060.0
HEART OF WICKER SMALL,4047.0


##Q3:	How much has each country spent on products?

In [0]:
mappedJob2 = invoices.map(lambda gross: (gross[7], (float(gross[3])) * (float(gross[5]))))
reducedJob2 = mappedJob2.reduceByKey(lambda x,y:x+y).filter(lambda mappedJob2: mappedJob2[0] is not None).sortBy(lambda sort: -sort[1]) 
print(reducedJob2.take(5))
GrossDollarAmount = reducedJob2.toDF(['Country','Gross'])
display(GrossDollarAmount.take(10))

[('United Kingdom', 965042.7499999746), ('EIRE', 29037.420000000002), ('Netherlands', 27200.859999999993), ('Germany', 22237.810000000005), ('France', 21773.329999999998)]


Country,Gross
United Kingdom,965042.7499999746
EIRE,29037.420000000002
Netherlands,27200.859999999993
Germany,22237.810000000005
France,21773.329999999998
Australia,9638.129999999997
Spain,8864.819999999998
Japan,7595.270000000001
Portugal,5694.299999999999
Switzerland,4909.549999999999


Output can only be rendered in Databricks

##Q4: What is the highest-grossing day in the dataset?  Quantity multiplied by unit price to get the revenue per line.

In [0]:
mappedJob3 = invoices.map(lambda highg: (highg[4][:7], float(highg[3]) * float(highg[5])))
print(mappedJob3.take(5))
reducedJob3 = mappedJob3.reduceByKey(lambda x, y:x+y).filter(lambda mappedJob3: mappedJob3[0] is not None).sortBy(lambda sort: -sort[1]) 
print(reducedJob3.take(5))
table = reducedJob3.toDF(['Date','highest-grossing_day'])
display(table.take(5))

[('12/1/10', 15.299999999999999), ('12/1/10', 20.34), ('12/1/10', 22.0), ('12/1/10', 20.34), ('12/1/10', 20.34)]
[('12/1/10', 58635.56000000026), ('12/2/10', 46207.27999999991), ('12/6/10', 53860.180000000015), ('12/8/10', 44189.839999999866), ('12/10/1', 57404.91000000017)]


Date,highest-grossing_day
12/1/10,58635.56000000026
12/2/10,46207.27999999991
12/6/10,53860.180000000015
12/8/10,44189.839999999866
12/10/1,57404.91000000017


In [0]:
GrossDollarAmount = reducedJob2.toDF(['Country','Gross'])
display(GrossDollarAmount.take(10))

Country,Gross
United Kingdom,965042.7499999746
EIRE,29037.420000000002
Netherlands,27200.859999999993
Germany,22237.810000000005
France,21773.329999999998
Australia,9638.129999999997
Spain,8864.819999999998
Japan,7595.270000000001
Portugal,5694.299999999999
Switzerland,4909.549999999999


Output can only be rendered in Databricks

In [0]:
table = reducedJob3.toDF(['Date','highest-grossing_day'])
display(table.take(30))

Date,highest-grossing_day
12/1/10,58635.56000000026
12/2/10,46207.27999999991
12/6/10,53860.180000000015
12/8/10,44189.839999999866
12/10/1,57404.91000000017
12/12/1,17240.92000000005
12/15/1,29443.69000000013
12/17/1,43534.19000000001
12/20/1,24741.750000000015
12/22/1,6134.569999999999


Output can only be rendered in Databricks